In [16]:
# Import stuff
from pandas import DataFrame

from nipype.pipeline.engine import Workflow, Node, MapNode, JoinNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import DataSink, FreeSurferSource, SelectFiles
from nipype.algorithms.misc import Gunzip

from nipype.interfaces.freesurfer.preprocess import MRIConvert
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer import FSCommand, MRIConvert, ReconAll
from nipype.interfaces.fsl.utils import Reorient2Std, MotionOutliers, Merge
from nipype.interfaces.fsl.preprocess import MCFLIRT, SliceTimer, FLIRT
from nipype.interfaces.fsl.maths import ApplyMask
from nipype.interfaces.fsl.model import GLM
from nipype.interfaces.fsl.epi import ApplyTOPUP, TOPUP
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.interfaces.nipy.preprocess import Trim

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Set study variables
analysis_home = '/home/camachocm2/Analysis/ChEC/fmri_proc'
raw_dir = analysis_home + '/raw'
preproc_dir = analysis_home + '/preproc'
firstlevel_dir = analysis_home + '/subjectlevel'
secondlevel_dir = analysis_home + '/grouplevel'
workflow_dir = analysis_home + '/workflows'
phase_encoding_file = analysis_home + '/misc/chec_encoding_file.txt'
mni_template = analysis_home + '/template/MNI152_T1_3mm_brain.nii.gz'
mni_brainmask = analysis_home + '/template/MNI152_T1_3mm_brain_mask.nii.gz'

#subject_info = DataFrame.read_csv(analysis_home + '/../misc/subjs.csv')
#subjects_list = subject_info['SubjID'].tolist()
subjects_list = ['pilot002','1003']

# FreeSurfer set up - change SUBJECTS_DIR 
fs_dir = analysis_home + '/freesurfer_dir'
FSCommand.set_default_subjects_dir(fs_dir)

# data collection specs
TR = 0.8 #in seconds
num_slices = 40
slice_direction = False #True = z direction, top to bottom
interleaved = True
echo_train_length = 0.068 # in seconds, loc (0x18,0x91) in the dcm header

In [17]:
## Data handling Nodes

# Select subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# FreeSurferSource - Data grabber specific for FreeSurfer data
fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')

# Sink data
substitutions = [('_subjid_', '')] #output file name substitutions
datasink = Node(DataSink(base_directory = preproc_dir,
                        container = preproc_dir,
                        substitutions = substitutions), 
                name='datasink')

## fMRI data Reconstruction Workflow

In [18]:
# Pull files
rawfile_template = {'folder': raw_dir + '/{subjid}/{func}'}
recon_selectfiles = Node(SelectFiles(rawfile_template), name='recon_selectfiles')
recon_selectfiles.iterables = [('func',['chec_movie_PA','corr_chec_1','corr_chec_2'])]

In [19]:
## reconstruction nodes

def sort_dicoms(dicom_directory):    
    from pydicom import dcmread
    from pandas import DataFrame
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from glob import glob
    from os.path import isdir
    from os import makedirs
    import shutil
    
    dicoms = glob(dicom_directory + '/MR.*')
    dicoms = sorted(dicoms)
     
    for dicom in dicoms:
        info = dcmread(dicom)
        te = info[0x18,0x81].value
        
        if not isdir(dicom_directory + '/TE_%d' % te):
                makedirs(dicom_directory + '/TE_%d' % te)

        shutil.move(dicom, dicom_directory + '/TE_%d' % te)
    
    dicom_names = []
    TE_folders = glob(dicom_directory + '/TE_*')
    TE_folders = sorted(TE_folders)
    for folder in TE_folders:
        temp = glob(folder + '/MR*')
        temp = sorted(temp)
        dicom_names.append(temp[0])
        
    return(dicom_names)


In [20]:
# Pull Dicom information
dicom_info = Node(Function(input_names=['dicom_directory'],
                           output_names=['dicom_names'],  
                           function=sort_dicoms), 
                       name='main_dicom_info')

# convert to niftis
convert_to_niftis = MapNode(MRIConvert(out_file='func.nii.gz', 
                                       out_type='niigz'), 
                            name='convert_to_niftis', 
                            iterfield=['in_file'])

In [21]:
recon_flow = Workflow(name='recon_flow')
recon_flow.connect([(infosource, recon_selectfiles,[('subjid','subjid')]), 
                    (recon_selectfiles, dicom_info,[('folder','dicom_directory')]), 
                    (dicom_info, convert_to_niftis,[('dicom_names','in_file')]),
                    
                    (convert_to_niftis, datasink, [('out_file','separated_niftis')])
                   ])

recon_flow.base_dir = workflow_dir
recon_flow.write_graph(graph2use='flat')
#recon_flow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

180626-10:08:22,88 workflow INFO:
	 Generated workflow graph: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/recon_flow/graph.png (graph2use=flat, simple_form=True).


'/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/recon_flow/graph.png'

## fMRI Data Preprocessing Workflows

In [22]:
## File handling nodes

pes_template={'PE_vol': preproc_dir + '/separated_niftis/{subjid}/_func_corr_{pe_dir}/_convert_to_niftis0/func.nii.gz'}
pes_selectfiles = Node(SelectFiles(pes_template), name='pes_selectfiles')
pes_selectfiles.iterables = ('pe_dir',['chec_1','chec_2'])

funcs_template={'func': preproc_dir + '/separated_niftis/{subjid}/_func_chec_movie_PA/_convert_to_{func_te}/func.nii.gz'}
funcs_selectfiles = Node(SelectFiles(funcs_template), name='funcs_selectfiles')
funcs_selectfiles.iterables = ('func_te',['niftis0','niftis1','niftis2'])

In [23]:
## Unwarping and ME cleaning nodes

# include only the first volume of each PE volume
trim_PEs = Node(Trim(end_index=1),name='trim_PEs')

# merge to 1 file for topup to calculate the fieldcoef
merge_pes = JoinNode(Merge(dimension='t',
                           merged_file='merged_pes.nii.gz'), 
                     name='merge_pes', 
                     joinsource='pes_selectfiles', 
                     joinfield='in_files')

topup = Node(TOPUP(encoding_file=phase_encoding_file), name='topup')

apply_topup = Node(ApplyTOPUP(in_index=[2], encoding_file=phase_encoding_file, 
                              method='jac', out_corrected='func_unwarped.nii.gz'), 
                   name='apply_topup')

In [24]:
prepreprocflow = Workflow(name='prepreprocflow')
prepreprocflow.connect([(infosource,pes_selectfiles, [('subjid','subjid')]),
                        (infosource,funcs_selectfiles, [('subjid','subjid')]),
                        (pes_selectfiles, trim_PEs, [('PE_vol','in_file')]), 
                        (trim_PEs, merge_pes, [('out_file','in_files')]), 
                        (merge_pes, topup, [('merged_file','in_file')]),
                        (topup, apply_topup, [('out_fieldcoef','in_topup_fieldcoef'), 
                                              ('out_movpar','in_topup_movpar')]),
                        (funcs_selectfiles, apply_topup, [('func','in_files')]),
                        (apply_topup, datasink, [('out_corrected','unwarped_funcs')])
                       ])

prepreprocflow.base_dir = workflow_dir
prepreprocflow.write_graph(graph2use='flat')
#prepreprocflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

180626-10:08:22,785 workflow INFO:
	 Generated workflow graph: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/prepreprocflow/graph.png (graph2use=flat, simple_form=True).


'/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/prepreprocflow/graph.png'

In [25]:
#file handling
preproc_template={'func': preproc_dir + '/unwarped_funcs/{subjid}/_func_te_niftis1/func_unwarped.nii.gz',
                  'anat': raw_dir + '/{subjid}/mprage.nii.gz'}
preproc_selectfiles = Node(SelectFiles(preproc_template), name='preproc_selectfiles')

In [26]:
## fMRI Data processing nodes

# reorient images to MNI space standard
reorientFunc = Node(Reorient2Std(terminal_output='file'),
                   name='reorientFunc')

# perform slice time correction
slicetime = Node(SliceTimer(index_dir=slice_direction,
                           interleaved=interleaved,
                           time_repetition=TR),
                name='slicetime')

# realignment using mcflirt
realignmc = Node(MCFLIRT(save_plots=True),
                name='realignmc')

# Coregistration using flirt
coregflt = Node(FLIRT(),
               name='coregflt')
coregflt2 = Node(FLIRT(apply_xfm=True, 
                       out_file='preproc_func.nii.gz'),
                name='coregflt2')

#unzip file before feeding into ART
gunzip_mask = Node(Gunzip(), name='gunzip_mask')
gunzip_func = Node(Gunzip(), name='gunzip_func')

# Artifact detection for scrubbing/motion assessment
art = Node(ArtifactDetect(mask_type='file',
                          parameter_source='FSL',
                          bound_by_brainmask=True,
                          norm_threshold=1,
                          zintensity_threshold=3,
                          use_differences=[True, False]),
           name='art')

# Register to MNI template
reg2mni = Node(FLIRT(reference=mni_template),
               name='reg2mni')
reg2mni2 = Node(FLIRT(apply_xfm=True, 
                      reference=mni_template,
                      out_file='proc_func.nii.gz'),
                name='reg2mni2')

In [27]:
## Anatomical processing

# Freesurfer autorecon1
proc_anat = Node(ReconAll(directive='autorecon1', 
                          flags='-gcut', subjects_dir=fs_dir), 
                 name='proc_anat')

# Convert skullstripped brain to nii, resample to 2mm^3
resample = Node(MRIConvert(out_type='niigz',
                          vox_size=(3,3,3)),
               name='resample')

# Reorient anat to MNI space
reorientAnat = Node(Reorient2Std(terminal_output='file'),
                   name='reorientAnat')

# Create binary mask of resampled, skullstripped anat, dilate, and erode to fill gaps
binarize = Node(Binarize(dilate=1,
                        erode=1,
                        invert=False,
                        min=1,
                        max=255),
               name='binarize')

In [32]:
# Data QC nodes
def create_coreg_plot(epi,anat):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    
    coreg_filename='coregistration.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'coregistration to anatomy')
    display.add_edges(anat)
    display.savefig(coreg_filename) 
    display.close()
    coreg_file = os.path.abspath(coreg_filename)
    
    return(coreg_file)

def check_mask_coverage(epi,brainmask):
    from os.path import abspath
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    from nipype.interfaces.nipy.preprocess import Trim
    
    trim = Trim()
    trim.inputs.in_file = epi
    trim.inputs.end_index = 1
    trim.inputs.out_file = 'epi_vol1.nii.gz'
    trim.run()
    epi_vol = abspath('epi_vol1.nii.gz')
    
    maskcheck_filename='maskcheck.png'
    display = plotting.plot_anat(epi_vol, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'brainmask coverage')
    display.add_contours(brainmask,levels=[.5], colors='r')
    display.savefig(maskcheck_filename)
    display.close()
    maskcheck_file = abspath(maskcheck_filename)

    return(maskcheck_file)

make_coreg_img = Node(name='make_coreg_img',
                      interface=Function(input_names=['epi','anat'],
                                         output_names=['coreg_file'],
                                         function=create_coreg_plot))

make_checkmask_img = Node(name='make_checkmask_img',
                      interface=Function(input_names=['epi','brainmask'],
                                         output_names=['maskcheck_file'],
                                         function=check_mask_coverage))
make_checkmask_img.inputs.brainmask=mni_brainmask

In [33]:
preprocflow = Workflow(name='preprocflow')

preprocflow.connect([(infosource, preproc_selectfiles, [('subjid','subjid')]),
                     (preproc_selectfiles,proc_anat, [('anat','T1_files')]),
                     (preproc_selectfiles,reorientFunc, [('func','in_file')]),
                     (infosource, proc_anat, [('subjid','subject_id')]),
                     (proc_anat, resample, [('brainmask','in_file')]),
                     (resample, reorientAnat, [('out_file','in_file')]),
                     (reorientAnat, binarize, [('out_file','in_file')]),
                     (reorientFunc, slicetime, [('out_file','in_file')]),
                     (slicetime, realignmc, [('slice_time_corrected_file','in_file')]),
                     (reorientAnat, coregflt, [('out_file','reference')]),
                     (realignmc, coregflt, [('out_file','in_file')]),
                     (realignmc, coregflt2, [('out_file','in_file')]),
                     (coregflt, coregflt2, [('out_matrix_file','in_matrix_file')]),
                     (reorientAnat, coregflt2, [('out_file','reference')]),              
                     (coregflt, reg2mni, [('out_file','in_file')]),
                     (coregflt2, reg2mni2, [('out_file','in_file')]),
                     (reg2mni, reg2mni2, [('out_matrix_file','in_matrix_file')]),                
                     
                     (binarize, gunzip_mask,[('binary_file','in_file')]), 
                     (gunzip_mask, art, [('out_file','mask_file')]),
                     (coregflt2, gunzip_func, [('out_file','in_file')]),
                     (gunzip_func, art, [('out_file','realigned_files')]),
                     (realignmc, art, [('par_file','realignment_parameters')]),
                     (coregflt, make_coreg_img, [('out_file','epi')]),
                     (reorientAnat, make_coreg_img, [('out_file','anat')]),
                     (reg2mni2, make_checkmask_img, [('out_file','epi')]),
                     
                     (reg2mni, datasink, [('out_file','reoriented_anat')]),
                     (make_checkmask_img, datasink, [('maskcheck_file','checkmask_image')]),
                     (make_coreg_img, datasink, [('coreg_file','coreg_image')]),
                     (reg2mni2, datasink, [('out_file','mnireg_func')]),
                     (art, datasink, [('plot_files','art_plot')]), 
                     (art, datasink, [('outlier_files','art_outliers')]),
                     (realignmc, datasink, [('par_file','mcflirt_displacement')])
                    ])
preprocflow.base_dir = workflow_dir
preprocflow.write_graph(graph2use='flat')
preprocflow.run('MultiProc', plugin_args={'n_procs': 1, 'memory_gb':8})

180626-10:48:05,756 workflow INFO:
	 Generated workflow graph: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/graph.png (graph2use=flat, simple_form=True).
180626-10:48:05,785 workflow INFO:
	 Workflow preprocflow settings: ['check', 'execution', 'logging', 'monitoring']
180626-10:48:05,834 workflow INFO:
	 Running in parallel.
180626-10:48:05,839 workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 8.00/8.00, Free processors: 1/1.
180626-10:48:05,937 workflow INFO:
	 [Node] Setting-up "preprocflow.preproc_selectfiles" in "/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_pilot002/preproc_selectfiles".
180626-10:48:05,949 workflow INFO:
	 [Node] Running "preproc_selectfiles" ("nipype.interfaces.io.SelectFiles")
180626-10:48:05,957 workflow INFO:
	 [Node] Finished "preprocflow.preproc_selectfiles".
180626-10:48:07,841 workflow INFO:
	 [Job 0] Completed (preprocflow.preproc_selectfiles).
180626-10:48:07,846 workflow INFO:
